In [2]:
texts=["The foxes are running fast"]

lemmas=[["the","fox","be","run","fast"]]

In [3]:
from datasets import Dataset

# Example dataset structure
data = {'text': texts, 'lemma': lemmas}
dataset = Dataset.from_dict(data)


In [4]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["text"], padding="max_length", truncation=True, is_split_into_words=True)
    
    labels = []
    for i, label in enumerate(examples["lemma"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Special tokens like [CLS] and [SEP] are ignored
            elif word_idx != previous_word_idx:
                label_ids.append(tokenizer.convert_tokens_to_ids(label[word_idx]))  # Lemma token ID
            else:
                label_ids.append(-100)  # Subtokens are ignored during loss calculation
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

ValueError: word_ids() is not available when using non-fast tokenizers (e.g. instance of a `XxxTokenizerFast` class).